In [1]:
import urllib.request
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd
from tqdm import tqdm

In [46]:
#Make header for dataframe & save file. Used the commented line only once.
#df = pd.DataFrame(columns=['1/4 Mile ET','1/4 Mile MPH','1/8 Mile ET','1/8 Mile MPH','0-60 Foot ET','VBOX 60-130 MPH','Standing Half Mile MPH','Standing 1 Mile MPH','Temperature F','Density Altitude (DA)','Timeslip Scan','VBOX Graph','VBOX 60-130 MPH.1','VBOX Graph.1','Empty','Horsepower @ wheels','Torque @ wheels','Dyno Type','Dyno Graph','Car Make','Car Model','Car Type','Car Year','Driver','E-Mail','Videos','Web Site','Weight with driver (lbs)','Track','Record Modified','Stock','Engine','Turbos','Supercharger','Boost Control','Boost Pressure (psi)','Intercooler','Nitrous Kit/System','Nitrous Shot','Pistons (rotors)','Port','Rings (seals)','Heads','Connecting Rods','Camshaft','Throttle Body','Intake & Air Filter','Headers  / Downpipe','Midpipe','Exhaust ','Carburetor','Computer ECU Chip Upgrade','Battery','Ignition','Spark Plugs','Fuel Injectors','Fuel Pump','Gas','Pulley','Transmission','Clutch/Torq Conv','Differential','Radiator','Flywheel','Shifter','Gears','Suspension','Wheels ','Brake Rotors/Pads','Tires'])
#df.to_csv('carlist.csv', encoding='utf-8')

In [ ]:
#Declares a dataframe from csv file.
df = pd.read_csv('carlist.csv')
df = df.drop(df.columns[0],axis=1)

In [2]:
#Wait is used throughout the script.
wait = 60 #60 second wait time

In [3]:
#Get URLs for each make in the scrape_page.
sleep(wait)
scrape_page = 'http://dragtimes.com/browse.php'
hdr = { 'User-Agent' : 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)' }
req = urllib.request.Request(scrape_page, headers=hdr)
response = urllib.request.urlopen(req)
soup = BeautifulSoup(response, 'html.parser')
make = soup.find_all('div', attrs={'class':'list-area'})

make_URLS = []
make1 = make[0].find_all('a')
for x in make1:
    make_URLS.append('http://dragtimes.com/'+x['href'])
del make_URLS[0]

In [4]:
#URLS for ALL records of the make
make_ALL = []
for x in make_URLS:
    sleep(wait)
    req = urllib.request.Request(x, headers=hdr)
    response = urllib.request.urlopen(req)
    soup = BeautifulSoup(response, 'html.parser')
    ALL = soup.find_all('div', attrs={'class':'list-area'})
    make_ALL.append('http://www.dragtimes.com/'+ALL[0].find_all('a')[0]['href'])

In [47]:
getcars= True
while getcars:
    try:
        # makedone.txt, makepagedone.txt, and carlistdoneurl.txt is used to load from previous session and runs where it lefted off.
        with open('makedone.txt') as f: 
            done_make = f.read()

        with open('makepagedone.txt') as f: 
            done_make_page = f.read()

        with open('carlistdoneurl.txt') as f: 
            done_cars = f.read()



        #Get max page number and make urls for each make pages

        for make in tqdm(make_ALL):
            if make in done_make:
                pass
            else:
                #sleep(wait)
                scrape_page = make
                #Header for request.
                hdr = { 'User-Agent' : 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)' }
                req = urllib.request.Request(scrape_page, headers=hdr)
                response = urllib.request.urlopen(req)
                soup = BeautifulSoup(response, 'html.parser')
                try:
                    max_page_nums = soup.find_all('center')[4].find_all('a')[-2].text
                except:
                    max_page_nums = 1

                Make_Page_URLS = []
                for pg in range(int(max_page_nums)):
                    Make_Page_URLS.append(scrape_page+'?resultpage='+str(pg+1))
                # Loop through each page and find each car link
                for make_page in Make_Page_URLS:
                    if make_page in done_make_page:
                        pass
                    else:
                        
                        sleep(wait)
                        #Header for request.
                        hdr = { 'User-Agent' : 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)' }
                        req = urllib.request.Request(make_page, headers=hdr)
                        response = urllib.request.urlopen(req)
                        soup = BeautifulSoup(response, 'html.parser')
                        links = soup.find('table', attrs={'class':'browse3table'}).find_all('a')
                        car_links = []
                        for x in links:
                            car_links.append('http://www.dragtimes.com/'+x['href'])
                        car_links = list(set(car_links))

                        #Loop through each car link and get the data
                        for y in car_links:
                            if y in done_cars:
                                pass
                            else:
                                pd.read_csv('carlist.csv')
                                sleep(wait)
                                hdr = { 'User-Agent' : 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)' }
                                req = urllib.request.Request(y, headers=hdr)
                                response = urllib.request.urlopen(req)
                                soup = BeautifulSoup(response, 'html.parser')
                                t3 = soup.find_all('span')
                                dflist = []
                                for x in t3[6:-1]:
                                    dflist.append(x.text)
                                hellodf = pd.DataFrame([dflist],columns=['1/4 Mile ET','1/4 Mile MPH','1/8 Mile ET','1/8 Mile MPH','0-60 Foot ET','VBOX 60-130 MPH','Standing Half Mile MPH','Standing 1 Mile MPH','Temperature F','Density Altitude (DA)','Timeslip Scan','VBOX Graph','VBOX 60-130 MPH.1','VBOX Graph.1','Empty','Horsepower @ wheels','Torque @ wheels','Dyno Type','Dyno Graph','Car Make','Car Model','Car Type','Car Year','Driver','E-Mail','Videos','Web Site','Weight with driver (lbs)','Track','Record Modified','Stock','Engine','Turbos','Supercharger','Boost Control','Boost Pressure (psi)','Intercooler','Nitrous Kit/System','Nitrous Shot','Pistons (rotors)','Port','Rings (seals)','Heads','Connecting Rods','Camshaft','Throttle Body','Intake & Air Filter','Headers  / Downpipe','Midpipe','Exhaust ','Carburetor','Computer ECU Chip Upgrade','Battery','Ignition','Spark Plugs','Fuel Injectors','Fuel Pump','Gas','Pulley','Transmission','Clutch/Torq Conv','Differential','Radiator','Flywheel','Shifter','Gears','Suspension','Wheels ','Brake Rotors/Pads','Tires'])
                                df = df.append(hellodf)
                                df.to_csv('carlist.csv', encoding='utf-8')
                                file = open('carlistdoneurl.txt','a')
                                file.write(str(y)+'\n')
                                file.close()
                        with open('makepagedone.txt','a') as f:
                            f.write("\n"+make_page)
                with open('makedone.txt','a') as f:
                    f.write("\n"+make)
        getcars = False
    except TimeoutError:
        print('timed out')
    except requests.HTTPError:
         print('url error')

 98%|█████████▊| 124/127 [1:46:41<18:43, 374.51s/it]/home/miko/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (8,18,27,29,31,32,34,36,38,39,40,41,42,43,44,45,46,47,48,49,50,51,53,54,56,57,59,61,62,63,64,65,66,67,68,69) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/miko/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (8,18,27,31,32,34,36,38,39,40,41,42,43,44,45,46,48,49,50,51,53,54,56,57,59,61,62,63,64,65,66,67,68,69) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/miko/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (8,27,31,34,36,38,39,41,42,43,45,46,48,49,50,51,53,54,56,57,59,61,62,63,64,65,66,67,68,69) have mixed types. Specify dtype optio